In [ ]:
# Add the parent directory to the path
import sys, os
sys.path.insert(0, os.path.abspath("../.."))

# Import numpy and ShortestPathGrid class
import numpy as np
from src.models.ShortestPathGrid import ShortestPathGrid

In [ ]:
# Generate data
m, n = (5, 5)  # grid size
# Create a random cost array for the grid
cost = np.arange((m-1)*n + m*(n-1))
np.random.shuffle(cost)
# Initialize the ShortestPathGrid with the cost array
grid = ShortestPathGrid(m, n, cost=cost)

In [ ]:
# Visualize the grid and edge costs
grid.visualize()

In [ ]:
# Set source and target of optimization
source = 1 # source node
target = 23 # target node

# Solve the shortest path problem
sol, obj = grid.solve(source=source, target=target)

In [ ]:
# Visualize the shortest path
grid.visualize(source=source, target=target, color_edges=sol)
print("Optimal objective:", obj)

In [ ]:
import pyepo

# Set parameters for data generation
num_train_data = 1000 # number of training data
num_test_data = 1000 # number of test data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width

# Generate data for shortest path problem
feats, costs = pyepo.data.shortestpath.genData(num_train_data+num_test_data, num_feat, (m,n), deg=deg, noise_width=e, seed=135)

In [ ]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test_data, random_state=42)

In [ ]:
from src.models.LinearRegression import LinearRegression

# Instantiate linear regression model
model = LinearRegression(num_feat=num_feat, num_edges=grid.num_edges())

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Choose loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# Data loading
train_data = torch.utils.data.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(c_train).float())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
test_data = torch.utils.data.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(c_test).float())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)

In [ ]:
from src.models.trainer import Trainer

# # 2) Build your two‐stage network:
# #    e.g. lr = LinearRegression(...); opt = ShortestPath(...)
# #    maybe wrap them in a single nn.Module
# model = MyOptNetPipeline(linear_model=model, opt_model=opt)

# Train linear regression model
trainer = Trainer(model, optimizer, criterion)
train_loss, test_loss = trainer.fit(train_loader, test_loader)


In [ ]:
import matplotlib.pyplot as plt

# Plot learning curve and optionally test loss
plt.plot(train_loss, label='Training Loss')
if test_loss:
    plt.scatter(np.arange(len(test_loss))*trainer.n_epochs, test_loss, marker='x', color='red', label='Testing Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
# init black box solver
spop = pyepo.func.SPOPlus(grid)

In [ ]:
# Test for a single data batch
x = object(); c = object(); cp = object(); w = object(); z = object()
for x_batch, c_batch in train_loader:
    x, c = x_batch, c_batch
    cp = model(x)
    w, z = grid.solve(source=source, target=target, cost=c)
    break

In [ ]:
spop(cp, c, w, z)